In [112]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK

In [114]:
Ti = 2
TiPrime = 2

In [115]:
Ti *2

4

In [116]:
dataset = "trains"

In [117]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [118]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [119]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

## TODO: define global variable and use inside of symjax for X placeholders
## TODO: 



In [120]:
rntk = RNTK(dic, 2, 2, X, n)

## LAYER ZERO ATTEMPT


In [121]:
tiprimes, tis = rntk.get_diag_indices()
jtiprimes, jtis = rntk.get_diag_indices(jnpbool = True)

2 0 1
2 0 1


## perform diagonal in parallel and scan in parallel

In [122]:
import jax.numpy as jnp

In [105]:
def sequential_diags(carry, idx, tiprimesph, tisph):
    print(tiprimesph[carry])
    print(carry)
    return carry+1, carry
    # return carry+1, rntk.diag_func_wrapper(tiprimesph[idx],tisph[idx], jmode = False)

In [110]:
T.arange(0, T.Variable(rntk.dim_num, "int32"))

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>
It arose in jax.numpy.arange argument `stop`.
While tracing the function call_wrapped at /Users/michaelsprintson/Documents/nosync.nosync/RNTK_UCI/RNTKVENV/lib/python3.7/site-packages/jax/linear_util.py:152, transformed by eval_shape., this concrete value was not available in Python because it depends on the value of the arguments to call_wrapped at /Users/michaelsprintson/Documents/nosync.nosync/RNTK_UCI/RNTKVENV/lib/python3.7/site-packages/jax/linear_util.py:152, transformed by eval_shape. at flattened positions [0], and the computation of these values is being staged out (that is, delayed rather than executed eagerly).
 (https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError)

In [106]:
diagindex = T.arange(0, rntk.dim_num)
diag = T.Variable((diagindex), "float32", "diag number")
diagtest = np.arange(0, rntk.dim_num)

In [107]:
all_ema = T.scan(sequential_diags, init = 0, sequences=[diag], non_sequences=[jtiprimes, jtis])

Op(name=_rewriting_take_1179, fn=_rewriting_take, shape=(), dtype=int32, scope=/default)
Placeholder(name=_134, shape=(), dtype=int32, scope=/default)


In [34]:
all_ema

Op(name=_scan[1], fn=_scan, shape=(5,), dtype=int32, scope=/default)

## perform diagonal in parallel but scan sequentially

In [124]:
array_of_diags=T.concatenate([rntk.diag_func_wrapper(i[0],i[1]) for i in zip(tiprimes, tis)])

In [125]:
full_lambda, full_phi = rntk.arrays_to_diag(array_of_diags)

In [126]:
print("LAMBDA")
print(full_lambda)
print("PHI")
print(full_phi)

LAMBDA
[[Op(name=_rewriting_take_1213, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_1217, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_1221, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]
 [Op(name=_rewriting_take_1225, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_1229, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_1233, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]
 [Op(name=_rewriting_take_1237, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_1241, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_1245, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]]
PHI
[[Op(name=_rewriting_take_1215, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/defa

## testing

In [19]:
bc = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(rntk.n, rntk.n) + (rntk.su ** 2)*X + rntk.sb ** 2 ## took out an X
single_boundary_condition = T.expand_dims(bc, axis = 0)
single_boundary_condition

Op(name=expand_dims_21, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [15]:
bc

Op(name=add_31, fn=add, shape=(10, 10), dtype=float32, scope=/default)

In [17]:
bc = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(rntk.n, rntk.n) + (rntk.su ** 2) + rntk.sb ** 2 ## took out an X
# single_boundary_condition = T.expand_dims(bc, axis = 0)
single_boundary_condition = T.expand_dims(T.Variable((bc), "float32", "boundary_condition"), axis = 0)

In [18]:
single_boundary_condition

Op(name=expand_dims_20, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [18]:
T.Placeholder((bc), "float32")

KeyboardInterrupt: 

In [15]:
single_boundary_condition = T.expand_dims(T.Placeholder((bc), "float32", "boundary_condition"), axis = 0)

KeyboardInterrupt: 

In [ ]:
single_boundary_condition